In [ ]:
import random
import os
os.environ["HF_HOME"] = r"./.cache"

from transformers import EncoderDecoderModel, AutoTokenizer, GenerationConfig, Seq2SeqTrainer, \
    Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from tokenizers import processors
from peft import LoraConfig, get_peft_model, TaskType
import evaluate

- Encoders
    - BERT_JA : `cl-tohoku/bert-base-japanese-v3`
    - BERT_EN : `bert-base-uncased`, `prajjwal1/bert-tiny`
- Decorders
    - GPT_JA : `rinna/japanese-gpt2-xsmall`
    - GPT_EN : `gpt2`

In [ ]:
SOURCE_LANG = "en"
RESUME = False
ckp = 25000

if SOURCE_LANG == "en":
    TARGET_LANG = "ja"
    encoder = "bert-base-uncased"
    decoder = "rinna/japanese-gpt2-small"
else: 
    TARGET_LANG = "en"
    encoder = "cl-tohoku/bert-base-japanese-v3"
    decoder = "gpt2"

ckp_path = f"./.ckp/{SOURCE_LANG}-{TARGET_LANG}-BERT-GPT2-xattn/checkpoint-{ckp}"
model = EncoderDecoderModel.from_pretrained(ckp_path, local_files_only=True)
model.cuda();

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_count, trainable_bytes = 0, 0
    total_count, total_bytes = 0, 0
    for _, param in model.named_parameters():
        total_count += param.nelement()
        total_bytes += param.nelement() * param.element_size()
        if param.requires_grad:
            trainable_count += param.nelement()
            trainable_bytes += param.nelement() * param.element_size()
    print(
        f"Total params: {total_count:12,} ({(total_bytes / 1024**2):7,.1f}MB) | "
        f"Trainable params: {trainable_count:12,} ({(trainable_bytes / 1024**2):7,.1f}MB) [{100 * trainable_count / total_count:3.1f}%]"
    )

In [ ]:
encoder_tokenizer = AutoTokenizer.from_pretrained(encoder, use_fast=True)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder, use_fast=True)
if decoder_tokenizer.pad_token_id is None:
    decoder_tokenizer.pad_token_id = decoder_tokenizer.eos_token_id

# add EOS token at the end of each sentence
decoder_tokenizer._tokenizer.post_processor = processors.TemplateProcessing(
    single="$A " + decoder_tokenizer.eos_token,
    special_tokens=[(decoder_tokenizer.eos_token, decoder_tokenizer.eos_token_id)],
)

In [ ]:
modules_to_save = ["LayerNorm", "ln_1", "ln_2", "ln_cross_attn", "ln_f"]
target_modules = ["word_embeddings", "position_embeddings", "token_type_embeddings", "query", "key", "value", "dense",
                    "wte", "wpe", "shared", "c_attn", "c_proj", "q_attn", "c_fc", "c_proj", "lm_head"]
config = LoraConfig(
    r=8,
    lora_alpha=8, # anything is fine (simply tune lr)
    lora_dropout=0.1,
    target_modules=target_modules,
    modules_to_save=modules_to_save,
    bias="lora_only",
    task_type=TaskType.SEQ_2_SEQ_LM
)
lora_model = get_peft_model(model, config)
# printing number of parameters the old way is broken!

In [ ]:
print_trainable_parameters(lora_model)

In [ ]:
from utils.dataset import EnJaDatasetMaker, EnJaDatasetSample, \
    OPUS100, JESC, MassiveTranslation, SnowSimplified, Tatoeba, IWSLT2017, \
    Flores
from utils.dataset.dataset_base import EnJaDataset

def get_csv_path(cls):
    assert issubclass(cls, EnJaDataset), "Invalid class passed!"
    return f"{EnJaDataset.DATASET_PROCESSED_DIR}/{cls.OUT_NAME}"

dataset = EnJaDatasetMaker.prepare_dataset(
    f"BERT-GPT2-{SOURCE_LANG}-{TARGET_LANG}",
    [
        EnJaDatasetSample(dataset=get_csv_path(OPUS100), nsample=50_000, ntokens=(0, 128)),
        EnJaDatasetSample(dataset=get_csv_path(JESC), nsample=150_000, ntokens=(0, 128)),
        EnJaDatasetSample(dataset=get_csv_path(MassiveTranslation), nsample=20_000, ntokens=(0, 128)),
        EnJaDatasetSample(dataset=get_csv_path(SnowSimplified), nsample=30_000, ntokens=(0, 128)),
        EnJaDatasetSample(dataset=get_csv_path(Tatoeba), nsample=125_000, ntokens=(0, 128)),
        EnJaDatasetSample(dataset=get_csv_path(IWSLT2017), nsample=175_000, ntokens=(0, 128)),
    ],
    source_language = SOURCE_LANG,
    model_type= "BERT-GPT2",
    encoder_tokenizer = encoder_tokenizer,
    decoder_tokenizer= decoder_tokenizer,
    num_proc  = 8,
    seed      = 123,
    splits    = (1, 0.002, 1) # rescaled to 1
)

train_data = dataset["train"].remove_columns(["source", "target"])

valid_data = Flores.load("dev").rename_columns({f"{SOURCE_LANG}_sentence": "source", f"{TARGET_LANG}_sentence": "target"})
valid_data = valid_data.map(
    EnJaDatasetMaker._get_map_compute_BERT_GPT2_tokenization(
        encoder_tokenizer=encoder_tokenizer, decoder_tokenizer=decoder_tokenizer
    )
)
valid_data = valid_data.remove_columns(["source", "target"])

data_collator = DataCollatorForSeq2Seq(encoder_tokenizer, model=model)

In [ ]:
from utils.metric import SacreBleu
compute_metrics = SacreBleu.get_mBART_metric(tokenizer=decoder_tokenizer, target_language=TARGET_LANG)

In [ ]:
def set_decoder_configuration(gc: GenerationConfig):
    gc.no_repeat_ngram_size = 4
    gc.length_penalty = 2.0
    gc.num_beams = 3
    #gen_config.max_new_tokens = MAX_LENGHT
    gc.max_length = 256
    gc.min_length = 0
    gc.early_stopping = True
    # pad token is set to eos since in GPT2 pad does not exist
    gc.pad_token_id = decoder_tokenizer.eos_token_id
    gc.bos_token_id = decoder_tokenizer.bos_token_id
    gc.eos_token_id = decoder_tokenizer.eos_token_id
    return gc

gen_config = GenerationConfig()
gen_config = set_decoder_configuration(gen_config)

In [ ]:
train_args = Seq2SeqTrainingArguments(
    report_to="wandb",
    run_name=f"{SOURCE_LANG}-{TARGET_LANG}-BERT-GPT2-xattn-LoRA",
    num_train_epochs=3,

    logging_strategy="steps",
    logging_steps=1, # * 4, 2, 1

    evaluation_strategy="steps",
    eval_steps=2500, # * 20_000, 10_000, 5_000
    prediction_loss_only=False,
    predict_with_generate=True,
    generation_config=gen_config,

    output_dir=f"./.ckp/{SOURCE_LANG}-{TARGET_LANG}-BERT-GPT2-xattn-LoRA/",
    save_strategy="steps",
    save_steps=2500, # * 20_000, 10_000, 5_000
    save_total_limit=100,
    load_best_model_at_end=True, # defaults to metric: "loss"
    metric_for_best_model="eval_score",
    greater_is_better=True,

    optim="adamw_torch",
    warmup_steps=400, # 3500, 1750, 875
    learning_rate=5e-5, # 3e-5, 5e-5
    bf16=True, # bf16, qint 8 ???
    
    group_by_length=True,
    length_column_name="length",

    # torch_compile=True,
    label_smoothing_factor=0.2, # 0.1, 0.2
    
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4, # * 1, 2, 4
    # gradient_checkpointing=True,
    # eval_accumulation_steps=4, # ???
)

In [ ]:
trainer = Seq2SeqTrainer(
    lora_model, 
    args=train_args,
    data_collator=data_collator,
    train_dataset=train_data, 
    eval_dataset=valid_data, 
    compute_metrics=compute_metrics
)

In [ ]:
lora_model.train()
trainer.train(resume_from_checkpoint=RESUME)